<a href="https://colab.research.google.com/github/mfilipak/AFRAC_IA/blob/main/002_Embeddings1_Portal_da_Transpar%C3%AAncia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EM CONSTRUÇÃO

# Experimento 002
##Objetivo: Experimentos com word embeddings usando como corpo o campo de descrição do dataset público do portal da transparência.
###Descrição: Experimentos iniciais para criação e visualização de word embeddings

Dica: No COLAB Use CTRL SPACE ao invés de TAB para "autocompletar". Ex:pd.re [CTRL SPACE] vai mostrar uma lista contendo as funções e atributos que começam com pd.re (como read_csv, ...) 

In [22]:
from glob import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random

In [3]:
#Copia os dados das NFEs do portal da cidadância pro drive virtual.
import requests  
file_url = "https://raw.githubusercontent.com//mfilipak/AFRAC_IA/main/DATASET/202201_NFe_NotaFiscalItem.zip"
r = requests.get(file_url, stream = True) 

with open("portal.zip", "wb") as file:  
    for block in r.iter_content(chunk_size = 1024): 
         if block:  
             file.write(block)

DATA_FILE = "portal.zip"
df = pd.read_csv(DATA_FILE, encoding="CP1252",sep=";")
print("O dataframe completo contém:",len(df),"linhas")

O dataframe completo contém: 324056 linhas


In [9]:
df3 = df[['DATA EMISSÃO','DESCRIÇÃO DO PRODUTO/SERVIÇO', 'CÓDIGO NCM/SH', 'CFOP']]
df3.columns = ["DATA", "DESCR", "NCM", "CFOP"]
df3 = df3[df3["NCM"]!=-1] #Filtrando NCMs = -1

text_lengths = np.array([len(_) for _ in df3['DESCR']])
df3 = df3[text_lengths>=3]

#Caso queira eliminar as repetições rodar a linha abaixo
df3 = df3.drop_duplicates(subset=["DESCR"])

['MAZA DILUENTE P/ EPOXI DP-003 5 L',
 'COLETOR DE MAT.PERFURO CORTANTES 07 LT',
 'Abóbora Cabotiá',
 'BECKER 50ML VIDRO BORO GRAD FB UNID C/ 1 PERFECTA',
 'DOCE JAZAM DUCREM NAPOLITANO',
 'COPO PLAST DESCARTAVEL 200ML INCOLOR COPOBRAS',
 'SAL REFINADO ZIZO 1KG',
 'CONCHA INOX',
 'MULTIFUNCIONAL BROTHER MFCL8900CDW',
 'Tinta para Carimbo Pilot Preta 42ml']

In [27]:
#No jargão do doc2vec, cada descrição é um document e o conjunto de docs um corpus
#Link: https://radimrehurek.com/gensim/auto_examples/core/run_core_concepts.html#sphx-glr-auto-examples-core-run-core-concepts-py
random.seed(42)
text_corpus = random.sample(list(df3["DESCR"]), 10)
text_corpus

['MAZA DILUENTE P/ EPOXI DP-003 5 L',
 'COLETOR DE MAT.PERFURO CORTANTES 07 LT',
 'Abóbora Cabotiá',
 'BECKER 50ML VIDRO BORO GRAD FB UNID C/ 1 PERFECTA',
 'DOCE JAZAM DUCREM NAPOLITANO',
 'COPO PLAST DESCARTAVEL 200ML INCOLOR COPOBRAS',
 'SAL REFINADO ZIZO 1KG',
 'CONCHA INOX',
 'MULTIFUNCIONAL BROTHER MFCL8900CDW',
 'Tinta para Carimbo Pilot Preta 42ml']